In [10]:
import pandas as pd
import numpy as np

from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [11]:
df = pd.read_csv(r'C:\Users\rodri\Desktop\Proyecto Reddit\Datos\final\datos_final.csv')

df.head(2)

,id,titulo,autor,fecha,url,upvotes,comentarios,conteo_palabras,contiene_palabras_clave
0,1f4u7nx,spotify says apple discontinued the tech for s...,hectordj18,2024-08-30 12:38:43,https://www.engadget.com/big-tech/spotify-says...,3374,375,14,1
1,1f50iq2,anandtech shuts down after years,atdoru,2024-08-30 17:09:03,https://www.theverge.com/2024/8/30/24232171/an...,909,111,5,0


In [12]:
# Procesamos los títulos (tokenización).
titulos = df['titulo'].apply(lambda x: x.split()) # Dividimos los títulos por palabras.

Vamos a entrenar un modelo Word2Vec para aprender las representaciones vectoriales de las palabras en los títulos.

In [13]:
# Entrenamos el modelo Word2Vec.
word2vec_modelo = Word2Vec(sentences = titulos, vector_size = 100, window = 5, min_count = 1, workers = 4)

# Verificamos que el modelo entrenó.
print(word2vec_modelo.wv.key_to_index) # Muestra las palabras del vocabulario.

{'to': 0, 'the': 1, 'of': 2, 'a': 3, 'in': 4, 'and': 5, 'for': 6, 'ai': 7, 'is': 8, 'on': 9, 'with': 10, 'new': 11, 'its': 12, 'after': 13, 'that': 14, 'are': 15, 'as': 16, 'it': 17, 'google': 18, 'us': 19, 'from': 20, 'at': 21, 'be': 22, 'over': 23, 'tech': 24, 'will': 25, 'how': 26, 'x': 27, 'by': 28, 'says': 29, 'data': 30, '$': 31, 'could': 32, 'about': 33, 'have': 34, 'not': 35, 'telegram': 36, 'down': 37, 'has': 38, 'you': 39, 'but': 40, 'microsoft': 41, 'power': 42, 'an': 43, 'ceo': 44, 'can': 45, 'this': 46, 'use': 47, 'now': 48, 'more': 49, 'apple': 50, 'california': 51, 'meta': 52, 'million': 53, 'was': 54, 'company': 55, '%': 56, 'out': 57, 'media': 58, 'accounts': 59, 'pavel': 60, 'hackers': 61, 'fake': 62, 'some': 63, 'elon': 64, 'app': 65, 'up': 66, 'no': 67, 'musks': 68, 'openai': 69, 'internet': 70, 'tesla': 71, 'used': 72, 'first': 73, 'big': 74, 'windows': 75, 'space': 76, 'do': 77, 'court': 78, 'battery': 79, 'social': 80, 'security': 81, 'against': 82, 'open': 83, '

Vamos a calcular el promedio de los embeddings de las palabras que contiene cada titulo. Esto nos va a dar una representación vectorial de cada título.

In [14]:
def titulo_embedding(titulo, modelo):

    # Obtenemos los vectores de las palabras y calculamos el promedio.
    vectores = [modelo.wv[word] for word in titulo if word in modelo.wv]

    if len(vectores) > 0:
        return np.mean(vectores, axis = 0)
    else:
        return np.zeros(modelo.vector_size)
    
# Convertimos cada título a un vector usando los embeddings de Word2Vec.
df['titulo_embedding'] = titulos.apply(lambda x: titulo_embedding(x, word2vec_modelo))

Ahora vamos a preparar los datos para el entrenamiento del modelo de regresión.

In [15]:
# Convertimos las listas de embeddings a un array para el entrenamiento.
X = np.stack(df['titulo_embedding'].values)

# Obtenemos las etiquetas (número de comentarios).
y = df['comentarios'].values

# Dividimos el dataset en conjuntos de entrenamiento y prueba.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [16]:
# Entrenamos el modelo.
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [17]:
# Hacemos predicciones.
y_pred = modelo.predict(X_test)

In [18]:
# Evaluamos el modelo.
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [10]:
# Mostramos los resultados.
print(f'Mean Squared Error: {mse}')
print(f'R2 Score: {r2}')

Mean Squared Error: 390719.00838866713
R2 Score: -0.1724463701248169


### Mean Squared Error (MSE): 390719.01

Este MSE es más alto que el del modelo del archivo 'modelo_caracteristicas.py', lo que nos indica que el modelo Word2Vec no está capturando tan bien la relación entre el título y el número de comentarios (si es que existe dicha relación).

### R² Score: -0.172

El R² Score también es negativo, indicando que el modelo no está mejorando sobre una simple media de comentarios.